In [5]:
curated_list = ['@AwkwardRambler','@AngryBlackN8V',
                '@Pandabbadon', '@LammaticHama','@DeadDogLake',
                '@cricketcrocker','@devilishgrin000','@LowArctic',
                '@ChelseyMooner','@akayatuk','@nativeopinion',
                '@BadSalishHeart','@joyem_braun','@RuthH_Hopkins',
                '@brettachapman','@NativeApprops','@Pam_Palmater',
                '@AzieDee','@BigIndianGyasi','@RoanhorseBex',
                '@Leave_Matoaka','@ZoeSTodd',
                '@notvanishing','@Dallas_Hunt','@justicedanielh',
                '@WordsandGuitar','@tuckeve','@tagaq',
                '@TanyaTalaga','@Hayden_King','@gindaanis',
                '@antalalakam','@dearnonnatives','@VinceSchilling',
                '@DelSchilling','@LowaBeebe',
                '@llewellynjobs','@DaveAlexRoberts','SaltyLilOjibwe',
                '@DarrylLeroux','@mredshirtshaw','@mors_lakota',
                '@Ruth4Nd','@Indigenia','@ItazipcoLakota',
                '@ryanredcorn','@KimTallBear','@tanayawinder',
                '@shotheekwe',
                '@msimmons444','@powhatannative','@aliwatson117',
                '@jfkeeler','@ThunderingElks',
                '@FrankWaln','@AgentNdn','@DeLesslin',]

In [6]:
print(len(curated_list))

57


In [7]:
# @justicedanielh, @KimTallBear on curated list also

scholar_list = ['@justicedanielh', '@PepePierce', '@pollysgdaughter', '@rebeccanagle', '@KimTallBear']

In [3]:
def get_tweet_object(search_term, start_date, end_date, num_of_tweets, include_retweets=True):
    
    '''
    takes date range, screen name & number of tweets
    
    search returns replies to input screen name
    
    returns tweets object
    
    input search term as a string
    
    input search range as datetime.datetime(20xx, 1, 20)
    
    include_retweets defaults is True. to exclude retweets set to False.
    
    '''
    
    
    search_args = searchtweets.load_credentials('twitter_keys.yaml',
                                                yaml_key='search_tweets_api',
                                                env_overwrite=False)
    
    if include_retweets:
        
        search_term = search_term + ' lang:en'
    
    else:
        
        search_term = search_term + ' lang:en -is:retweet'
        
    
    
    rule = searchtweets.gen_rule_payload(search_term,
                                         results_per_call=500,
                                         from_date=start_date.strftime('%Y-%m-%d'),
                                         to_date=end_date.strftime('%Y-%m-%d'))     
        
    tweet_objects = []
    
    rs = searchtweets.ResultStream(rule_payload=rule,
                                   max_results=num_of_tweets,
                                   max_pages= num_of_tweets / 500,
                                   **search_args)
    
    start_time = time.time()

    for i, tweet_object in enumerate(rs.stream(), start=1):
        
        if i % 500 == 0:
            
            now = time.time()
            
            if now < start_time + 4:
                
                time.sleep(4 - (now - start_time))
                
                start_time = time.time()
                
        tweet_objects.append(tweet_object)
    
    return tweet_objects

def tweet_obj_to_df(tweet_objects):
    
    tweet_dict = {'text': [],
                  'author': []
                 }
    
    for tweet in tweet_objects:
        
        tweet_dict['text'].append(tweet.all_text)
        
        tweet_dict['author'].append(tweet.screen_name)
    
    tweets_df = pd.DataFrame(tweet_dict)
    
    return tweets_df